**Sentinel allows customers to implement policy-as-code in the same way that Terraform implements infrastructure-as-code.**

**The Sentinel Command Line Interface (CLI) allows you to apply and test Sentinel policies including those that use mocks generated from Terraform Cloud and Terraform Enterprise.**

**TFC Free plans support 1 Policy Set and 5 Policies: https://www.hashicorp.com/products/terraform/pricing**

The first thing we are going to do in this exercise is to sync a VCS repo containig Terraform code with Terraform Cloud. To that end, we are going to use a local terraform workspace to create the TFC Workspace that will create infrastructure in AWS and against which we are going to test our Sentinel policies.

### **variables.tfvars**
| Variable Name   | Description              |
| --------------- | ------------------------ |
| organization    | TFC Organization name    |
| project         | Project name             |
| gh_token        | Github OAuth Token       |
| repo            | Github repo              |
| working_dir     | Relative path where TF code is placed |

### **Create TFC Workspace associated to VCS repo**

In [6]:
%%bash
cd tf-config-tfe
terraform init
terraform apply -var-file=variables.tfvars -auto-approve


Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/tfe from the dependency lock file
- Using previously-installed hashicorp/tfe v0.48.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # tfe_project.sentinel_test_project will be created
  + resource "tfe_project" "sentinel_test_project" {
      + id           = (known after apply)
      + name         = "tf

![Alt text](image.png)
![Alt text](image-1.png)

Now that we have a working Terraform configuration and workspace, lets create a policy to set guardrails around one of type of resources created by our config. We want to enforce 2 organization requirements for S3 buckets:

* All S3 buckets should have `department` and `environment` tags
* All S3 bucket should use the `private` ACL to prevent accidental data leaks

## Sentinel Imports

A Sentinel policy can include imports which enable a policy to access reusable libraries, external data and functions. Terraform Cloud provides four built-in imports that can be used for a policy check:

| Import  | Description |
| ------- | ----------- |
| [tfplan](https://developer.hashicorp.com/terraform/cloud-docs/policy-enforcement/sentinel/import/tfplan-v2)  |	provides access to Terraform plan details which represent the changes Terraform will make to the desired state |
| [tfconfig](https://developer.hashicorp.com/terraform/cloud-docs/policy-enforcement/sentinel/import/tfconfig-v2) |	provides access to Terraform configuration that is being used to describe the desired state |
| [tfstate](https://developer.hashicorp.com/terraform/cloud-docs/policy-enforcement/sentinel/import/tfstate-v2) |	provides access to Terraform statewhich represents what Terraform knows about the real world resources |
| [tfrun](https://developer.hashicorp.com/terraform/cloud-docs/policy-enforcement/sentinel/import/tfrun) |	provides access to information about a run |

Note: Some imports can have a v2 suffix which indicates they represent the new data structures used post Terraform 0.12

We have created an example sentinel policy in `restrict-s3-buckets.sentinel`. Let's analyze each of its parts:
```bash

import "tfplan/v2" as tfplan

# Filter S3 buckets
s3_buckets = filter tfplan.resource_changes as address, rc {
  rc.type is "aws_s3_bucket" and
  (rc.change.actions contains "create" or rc.change.actions is ["update"])
}
```
* `tfplan/v2` is a frequently used import in policies since it provides details about planned changes. Later in this lab we will go over how you can determine what information is available to your policy from the import.
* `tfplan.resource_changes` is a collection with the resource address as the key and a resource change object as the value.
We are iterating over each of the resource change objects in the collection and using the filter function to filter out change objects whose type is `aws_s3_bucket`.
* The `type` name matches the resource block we would define in a .tf file to manage S3 buckets.

To understand where is that what we are checking let's run a `Plan` on the TFC workspace and get some `mock` data.

In [8]:
%%bash
# SE only
doormat login 
doormat aws --account aws_jose.merchan_test --tf-push --tf-workspace  Sentinel --tf-organization josemerchan-training

time="2023-10-04T17:39:24+02:00" level=info msg="Found new version v4.0.0, checking compatibility"
time="2023-10-04T17:39:25+02:00" level=info msg="Version 4.0.0 is compatible; preparing to download"
Downloading...  100% |█████████████████████████| (27/27 MB, 6.014 MB/s)        
time="2023-10-04T17:39:30+02:00" level=info msg="Downloaded release archive with shasum 748a33c56379c4f1c088654e5a2ceb8a1decc4f33f588bb0707263bb5f170288"
time="2023-10-04T17:39:31+02:00" level=info msg="Verified the signature of the release shasums file and verified the shasum of the downloaded archive"
time="2023-10-04T17:39:31+02:00" level=info msg="Decompressed new doormat binary to /opt/homebrew/Cellar/doormat-cli/3.3.0/bin/doormat_4.0.0"
time="2023-10-04T17:39:31+02:00" level=info msg="Backed up current doormat-cli binary to /opt/homebrew/Cellar/doormat-cli/3.3.0/bin/doormat.v3.6.2.bak"
time="2023-10-04T17:39:33+02:00" level=info msg="Updated doormat-cli from v3.6.2 to v4.0.0"
time="2023-10-04T17:39:33+02:

Let's Create a Plan-only run


![Alt text](image-2.png)

A plan will be queued. Once finished it will give use what is planned to be changed. Additionally it gives us the option to `Download Sentinel mocks` 

![Alt text](image-3.png)

You can download them or simply use the sample mocks provided within this repo, within the /S3-demo/mocks directory. The one we care about at this point is `mock-tfplan-v2.sentinel`

The file contains a number of top level objects (`collections`) that can be iterated

| file example             | doc reference            |
| -----------------------  | -------------            |
| ![Alt text](image-4.png) | ![Alt text](image-5.png) |

So back to our example we are iterating over resource changes. If you look at the plan output above there are 6 resources that "will be created". Those 6 all also the resources within the resource_changes list

![Alt text](image-6.png)


```bash

import "tfplan/v2" as tfplan

# Filter S3 buckets
s3_buckets = filter tfplan.resource_changes as address, rc {
  rc.type is "aws_s3_bucket" and
  (rc.change.actions contains "create" or rc.change.actions is ["update"])
}
```

* From the collection the only one that matches is the first resource: `aws_s3_bucket.dev`
* The other thing we are doing is apply and `AND` and verify that the kind of change apply to the resource in this case whether the resource is `create`(d) or `update`(d)

| ![Alt text](image-7.png) |  ![Alt text](image-8.png) |

The resource `aws_s3_bucket.dev` matches both conditions. The result is the creation of a list (or based on the [documentation](https://docs.hashicorp.com/sentinel/language/collection-operations#filter-expression):  `a subset of the provided collection` ) with a single element on it

### Rules

The next steps is check that our collection (the subset of it based on the `filter` expression) matches the characteristics we want to enforce:
* All S3 buckets should have `department` and `environment` tags
* All S3 bucket should use the `private` ACL to prevent accidental data leaks

1. Let's focus on the first of those. 

```bash
# The tags we want to enforce
required_tags = ["department", "environment"]

# Lets create a subset/list if it contains tags but those tags are not in the "required_tags" list
tag_violators = filter s3_buckets as address, bucket {
  any required_tags as rtag {
    rtag not in bucket.change.after.tags
  }
}
# Given a list of aws_s3_bucket resources with invalid tags, validate if the list is empty.
# If not empty the rule evaluate to 'false' (fail), but if empty then evaluate to 'true' (pass)
bucket_should_have_required_tags = rule {
  tag_violators is empty
}
```

Let's go back to our file in `S3-demo/mocks/mock-tfplan-v2.sentinel`. 

The resource has the `tag` **environment** but not **department**. For this reason this [rule](https://docs.hashicorp.com/sentinel/intro/getting-started/rules) will result in a `FAIL` when evaluated.



```json
	"aws_s3_bucket.dev": {
		"address": "aws_s3_bucket.dev",
		"change": {
			"actions": [
				"create",                             # action create
			],
			"after": {
				"acl":                       "private",
				"bucket_prefix":             null,
				"cors_rule":                 [],
				"force_destroy":             true,
				"grant":                     [],
				"lifecycle_rule":            [],
				"logging":                   [],
				"policy":                    null,
				"replication_configuration": [],
				"tags": {
					"environment": "dev",                # Just enviroment, department is missing
				},
				"tags_all": {
					"environment": "dev",
				},
				"website": [],
			},
			"after_unknown": {
				...
			},
			"before": null,
		},
		"deposed":        "",
		"index":          null,
		"mode":           "managed",
		"module_address": "",
		"name":           "dev",
		"provider_name":  "registry.terraform.io/hashicorp/aws",
		"type":           "aws_s3_bucket",             # aws_s3_bucket resource
	},

```


1. Let's move onto the second one, making sure that the acl policy associated to the bucket is set to `private`

```bash
# Given the collection of resources in the tfplan iterate and obtain those whose type is aws_s3_bucket_acl and that are going to be created or updated
s3_bucket_acls = filter tfplan.resource_changes as address, rc {
  rc.type is "aws_s3_bucket_acl" and
  (rc.change.actions contains "create" or rc.change.actions is ["update"])
}

# For the collection of aws_s3_bucket_acl get those whose change.after.acl is not private
acl_violators = filter s3_bucket_acls as address, bucket {
  bucket.change.after.acl != "private"
}
# If the subset above is not empty then result in FAIL
bucket_acl_should_be_private = rule {
  acl_violators is empty
}
```

Again, let's go back to our file in `S3-demo/mocks/mock-tfplan-v2.sentinel`. Here we can find a resource that matches both conditions

```json
	"aws_s3_bucket_acl.dev": {
		"address": "aws_s3_bucket_acl.dev",
		"change": {
			"actions": [
				"create",                      # action create
			],
			"after": {
				"acl": "public-read",           # is not private
				"expected_bucket_owner": null,
			},
			"after_unknown": {
				"access_control_policy": true,
				"bucket":                true,
				"id":                    true,
			},
			"before": null,
		},
		"deposed":        "",
		"index":          null,
		"mode":           "managed",
		"module_address": "",
		"name":           "dev",
		"provider_name":  "registry.terraform.io/hashicorp/aws",
		"type":           "aws_s3_bucket_acl",  # aws_s3_bucket_acl
	},
````
